Multi-Objective Traveling Salesman Problem 

# Definicja Problemu: 

Podstawowy problem TSP polega na optymalizacji ścieżki którą nasz handlaż podróżuje, której celem jest znalezienie takiej permutacji odwiedzanych przez niego miast, żeby ścieżka którą podróżuje była najkrótsza. Problem który w tej pracy zamierzam rozwiązać jest mocno powiązany z jego poprzednią wersją. Mianowicie dalej istnieje handlarz poruszający się po n-miastach jednak tym razem zamiast minimalizować tylko drogę musi również zadbać o inne rzeczy, np koszty,ślad-węglowy. Niestety mocno komplikuje to interpretacje 'optymalnego' rozwiązania o czym więcej w dalszej części. Podsumowując Multi-Objective TSP będzie polegał na znalezieniu grupy rozwiązań takich, że użytkownik jest w stanie wybrać które z celów są dla niego najważniejsze, czyli w rozwiązaniach powinny się znaleźć rozwiązania minimalizujące każdą z funkcji celu z osobna, ale również takie które biorą pod uwagę ich podzbiór w tym również takie uśredniające wszystkie cele. 

# ROZWAŻANIA NAD OPTYMALNYM ROZWIAZANIEM:

Jeśli zastanowimy się chwilę nad powyższą definicją, ma ona pewien problem, mianowicie ciężko jest proównywać ze sobą rozwiązania. W przypadku jednej funkcji celu było to bardzo proste i polegało to jedynie na obliczaniu funkcji celu z pierwszego a potem drugiego rozwiązania i operowaniu na zwykłym porządku liczb rzeczywistych. W tym przypadku jednak nie można tak zrobić ponieważ gdybyśmy ekstapolowali tą logikę do $R^k$ gdzie k- liczebność funkcji celu, to bazowy porządek liniowy $R^k$ wprowadza nam hierarchię funkcji celu (narzuca która funkcja jest ważniejsza) tak, że pozostałe są przydatne jedynie do rozstrzygania 'remisów'.

Przykład: mamy dwie funkcje cely f i g. w takim przypadku dla każdego osobnika x z populacji P tworzymy wektor (f(x),g(x)) należący do $R^2$.
stosując jednak liniowy porządek $R^2$ gdy porównamy $x_1$ z $x_2$. To $ f(x_1) <  f(x_2)  \Rightarrow  x_1 < x_2$,

funkcje g rozpatrzymy dopiero gdy $ f(x_1) == f(x_2) $ bo wtedy $ g(x_1) < g(x_2) \Rightarrow x_1 < x_2 $

WNIOSEK 1:

Musimy więc wprowadzić inny porządek który pozwoli nam w jakiś sposób porównywać ze sobą poszczególne rozwiązania

WNIOSEK 2:

Istnieje sytuacja w której niezależnie od porządku który chcemy ustalić, jesteśmy wstanie wybrać które rozwiązanie jest 'lepsze'.

Gdy $x_1$ osiąga bardziej pożądane lub równe wyniki dla każdej funkcji celu od $x_2$ tak że conajmniej dla jednej z funkcji zachodzi $f(x_1)<f(x_2)$ (gdy szukamy minimum) wtedy możemy jednoznacznie powiedzieć że $x_1$ jest lepszym rozwiązaniem.

# Idea 1:
Utworzenie funkcji $R^k \to R$  i stosowanie standardowego porządku na R.

Taką funkcją będzie norma $R^k$ dla znormalizowanych funcji. Dlaczego właśnie tak? Chcemy znormalizować funkcje ponieważ chcemy żeby rózne funkcje równie mocno na nią wpływały. 


W poniższej implementacji $k=2$ i $n=10$ żeby czas sprawdzenia wszystkich kombinacji nie był zbut długi. Dane bedą dwoma macierzami symetrycznymi A,B takimi że $A_{i,j}$ - długość drogi z miasta i do j, natomiast $B_{i,j}$ - koszt drogi z miasta i do j.

In [5]:
import numpy as np
import data_mo_tsp


'''
# domyślnie każda krawędź jest ograniczona przez 100
odleglosci,koszty = data_mo_tsp.generate_data(10,2)
np.save('distances',odleglosci)
np.save('costs',koszty)
'''

costs = np.load('costs.npy')
odleglosci = np.load('distances.npy')


def dist_goal_fun(perm):
    curr_pos = 0
    dist=0
    for i in perm:
        dist+=odleglosci[curr_pos][i]
        curr_pos=i
    dist+=odleglosci[curr_pos][0]
    return dist

def cost_goal_fun(perm):
    curr_pos = 0
    cost = 0
    for i in perm:
        cost+=costs[curr_pos][i]
        curr_pos=i
    cost+=costs[curr_pos][0]
    return cost

def eval(perm):
    return np.array([dist_goal_fun(perm),cost_goal_fun(perm)])


Brute by dowiedzieć się jakie są 'optymalne' rozwiązania. (min kosztów , min trasa , średnie koszty średnia trasa)

In [4]:
import itertools

#tylko 9 bo zawsze zaczynamy w 1
indeksy = list(range(9))
# koszt maksymalny  = max_krawedź (100) razy ilość miast (10)
perm_min_cost = 1001
cost_perm = None
perm_min_dist = 1001
dist_perm = None
# max norm = pierwiastek sumy kwadratów max z cost i dist
perm_min_norm = 2000
norm_perm = None
for i in itertools.permutations(indeksy):
    perm = np.array(i)+1
    vec2 = eval(perm)
    norm = np.linalg.norm(vec2)
    if(perm_min_dist>vec2[0]):
        dist_perm=perm
        perm_min_dist = vec2[0]
    if(perm_min_cost>vec2[1]):
        cost_perm=perm
        perm_min_cost = vec2[1]
    if(perm_min_norm>norm):
        norm_perm=perm
        perm_min_norm=norm

print(f'min z kosztów to: {perm_min_cost} obliczony przez kolejność: {cost_perm}')
print(f'min z dystansów to: {perm_min_dist} obliczony przez kolejność: {dist_perm}')
print(f'min z normy to: {perm_min_norm} obliczony przez kolejność: {norm_perm}')
np.save('opt_cost',perm_min_cost)
np.save('opt_cost_perm',cost_perm)
np.save('opt_dist',perm_min_dist)
np.save('opt_dist_perm',dist_perm)
np.save('opt_norm',perm_min_norm)
np.save('opt_norm_perm',norm_perm)



min z kosztów to: 282 obliczony przez kolejność: [3 2 1 4 8 6 5 7 9]
min z dystansów to: 267 obliczony przez kolejność: [1 4 9 5 6 2 8 3 7]
min z normy to: 475.47870614781476 obliczony przez kolejność: [7 5 6 2 1 4 3 8 9]


## przemyślenie po zobaczeniu wyników

Odrazu po tych wynikach spodziewamy się że jest coś nie tak, ponieważ każdy z nich zwrócił różne rozwiązanie (bardzo różne) więc prawdopodobnie norma nie jest dobrym wyznacznikiem porządku. (Istnieją sytuacje żeby była [kiedy opt wszystkich funkcji celu to ta sama permutacja] jednak w ogólności tak nie jest. Jednak i tak jest w czymś pomocna). Słowem komentarza jeszcze dlaczego nie normalizowaliśmy funkcji przed liczeniem normy, zrobiliśmy tak dlatego że max dla obu tych funkcji celu był by taki sam. Dodatkowo obie funkcje cely rosną w podobny sposób więc nie mamy problemu postaci drobna optymalizacja jednej funkcji spowoduje różniącą sie o znaczący rząd wielkości zmianę w drugiej.

## eksperyment 1 (Norma)

In [3]:
class genetic_tsp_norm():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def eval_pop(self):
        return np.argsort([np.linalg.norm(eval(perm)) for perm in self.population])

    def step(self):
        sorted=self.eval_pop()
        self.population = self.population[sorted][:(self.pop_size//2)]
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))



model = genetic_tsp_norm(eval,10,100,1)
for i in range(1000):
    model.step()

for i in range(10):
    perm = model.population[i]
    vec2 = eval(model.population[i])
    norm = np.linalg.norm(vec2)
    print(f'{i}-ta z kolei permutacja - {perm} o wektorze wynikowym {vec2} osiągnęła norme {norm}')



0-ta z kolei permutacja - [7 5 6 2 1 4 3 8 9] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
1-ta z kolei permutacja - [7 5 6 2 1 4 3 8 9] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
2-ta z kolei permutacja - [7 5 6 2 1 4 3 8 9] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
3-ta z kolei permutacja - [9 8 3 4 1 2 6 5 7] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
4-ta z kolei permutacja - [7 5 6 2 1 4 3 8 9] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
5-ta z kolei permutacja - [9 8 3 4 1 2 6 5 7] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
6-ta z kolei permutacja - [9 8 3 4 1 2 6 5 7] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
7-ta z kolei permutacja - [9 8 3 4 1 2 6 5 7] o wektorze wynikowym [324 348] osiągnęła norme 475.47870614781476
8-ta z kolei permutacja - [9 8 3 4 1 2 6 5 7] o wektorze wynikowym [324 348] osiągnęła norme 475.4787061

## wnioski po eksperymencie 1

Widzimy że wyniki są nie najgorsze pod względem tego że algorytm znalazł optymalną norme, dodatkowo widzimy że wektor funkcji celu ma wyniki pośrednie, nie osiąga on żadnego z minimów ale za to każda z 2 wartości funkcji jest mniejsza niż wartość drugiej funkcji w permutacji minimalnej

Wniosek: Do dlaszych obliczeń będziemy brali pod uwagę norme po to by znajdywać wyniki pośrednie

# Idea 2:
Zamiast wprowadzać jakiś porządek na R^k będziemy losowali jedną z funkcji i kolejną generacje tworzyli względem tej własnie.
Dlaczego miało by działac, przy odpowiednio dużej liczbie iteracji każda z funkcji powinna mieć mniej więcej tyle samo generacji stworzonych względem samej siebie, z założeniem takim że przynajmniej fragment optymalnej populacji funkcji "przeżywa" ocenianie względem innej funkcji. (Jak można to sobie wyobrażać: mamy kilka punktów 'zbierzności' czyli punktów reprezentujących optymalne wartości względem którejś z funkcji, teraz startujemy w losowych rozmiar populacji miejscach, i z każdą iteracją punkty blisko punktu zbierzności związanego z wylosowaną funkcją powinny się do niego zbliżyć jednocześnie zostawiając przy życiu punkty blisko pozostałych funkcji) 

## eksperyment 1 (losowa funkcja ewaluacji)

In [9]:
class genetic_tsp_random_fun_eval():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        self.WhichIsEvaledMore = np.zeros(2)
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def eval_pop(self):
        if(np.random.uniform()<0.5):
            self.WhichIsEvaledMore[0]+=1
            return np.argsort([eval(perm)[0] for perm in self.population])
        else:
            self.WhichIsEvaledMore[1]+=1
            return np.argsort([eval(perm)[0] for perm in self.population])

    def step(self):
        sorted=self.eval_pop()
        self.population = self.population[sorted][:(self.pop_size//2)]
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))

model = genetic_tsp_random_fun_eval(eval,10,100,1)
for i in range(1000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(vec2[0]<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
print('100 osobnikow -----------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('-------------------------------------')
model = genetic_tsp_random_fun_eval(eval,10,1000,1)
for i in range(1000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(vec2[0]<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm

print('1000 osobnikow  --------------------------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('----------------------------------------------------------')     

[534. 466.]
100 osobnikow -----------------------
Min dystans: 267 osiagniety przez: [7 3 4 9 5 6 2 8 1]
Min koszt: 373 osiagniety przez: [9 5 6 4 8 1 2 3 7]
Min norma: 624.0576896409498 osiagniety przez: [1 5 9 4 6 2 8 3 7]
-------------------------------------
[509. 491.]
1000 osobnikow  --------------------------------------
Min dystans: 267 osiagniety przez: [7 3 4 9 5 6 2 8 1]
Min koszt: 342 osiagniety przez: [9 8 3 7 5 6 2 4 1]
Min norma: 672.8803756983851 osiagniety przez: [1 3 8 7 4 9 5 6 2]
----------------------------------------------------------


## eksperyment 1 b) (losowa funkcja ewaluacji + norma)

Teraz powtórzymy eksperyment ale mozliwa do wylosowania bedzie jeszcze evaluacja po normie

In [14]:
class genetic_tsp_random_fun_eval():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        self.WhichIsEvaledMore = np.zeros(3)
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def eval_pop(self):
        zmienna_losowa=np.random.uniform()
        if(zmienna_losowa<0.33):
            self.WhichIsEvaledMore[0]+=1
            return np.argsort([eval(perm)[0] for perm in self.population])
        if(zmienna_losowa<0.66):
            self.WhichIsEvaledMore[1]+=1
            return np.argsort([eval(perm)[0] for perm in self.population])
        else:
            self.WhichIsEvaledMore[2]+=1
            return np.argsort([np.linalg.norm(eval(perm)) for perm in self.population])

    def step(self):
        sorted=self.eval_pop()
        self.population = self.population[sorted][:(self.pop_size//2)]
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))

model = genetic_tsp_random_fun_eval(eval,10,100,1)
for i in range(1000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(vec2[0]<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
print('100 osobnikow -----------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('-------------------------------------')
model = genetic_tsp_random_fun_eval(eval,10,1000,1)
for i in range(1000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(vec2[0]<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm

print('1000 osobnikow  --------------------------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('----------------------------------------------------------') 

[335. 333. 332.]
100 osobnikow -----------------------
Min dystans: 267 osiagniety przez: [1 8 2 6 5 9 4 3 7]
Min koszt: 352 osiagniety przez: [9 7 5 6 8 2 1 4 3]
Min norma: 556.2032002784593 osiagniety przez: [9 5 6 2 8 3 4 1 7]
-------------------------------------
[330. 331. 339.]
1000 osobnikow  --------------------------------------
Min dystans: 269 osiagniety przez: [7 3 4 9 5 6 8 2 1]
Min koszt: 337 osiagniety przez: [7 5 6 2 1 4 8 3 9]
Min norma: 558.5875043357129 osiagniety przez: [1 4 3 7 2 8 6 5 9]
----------------------------------------------------------


## wnioski z eksperymentu 1

Jak widzimy po wynikach Funkcje były losowane mniej więcej równomiernie jednak w wygenerowanych wynikach osiągamy tylko jedno minimum czyli nasze założenie było nie trafne, i osobniki po wylosowaniu innej fukncji niz tak którą optymalizują wymierają. Pomimo zwiekszenia populacji dalej ten problem istnieje. W teorii zwiększenie populacji powinno roziwazac ten problem ponieważ odrzucamy tylko pewną ilość osobnikow jednak najwyraźniej nie jest to na tyle skuteczne przy wylosowaniu tej samej funkcji kilka razy pod rząd. 

By rozwiązać ten problem wprowadzę miejsca zarezerwowane, będą one polegaly na tym że w każdej populacji po ewaluacji w zgledem wybranej funkjci najpiwer wrzucamy do nowej populacji optima pozostałych funkcji a potem względem kolejności wylosowanej funkcji. Przy czym rezerwacja bedzie uwzgledniana tylko w populacji rodziców nie dzieci żeby nie utracić naszej zdolności przeszukiwania przestrzeni tylko żeby zachować pożądane osobniki.

## eksperyment 2 (miejsca zarezerwowane)

In [13]:
class genetic_tsp_random_fun_eval_reserved():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty,reservation_size=10):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        self.WhichIsEvaledMore = np.zeros(3)
        self.reservation_size = reservation_size
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def eval_pop(self):
        return np.array([self.eval_fun(perm) for perm in self.population])

    def step(self):
        pop_evaluation=self.eval_pop()
        sorted0 = np.argsort(pop_evaluation[:, 0]) # by pop_evaluation[i][0]
        sorted1 = np.argsort(pop_evaluation[:, 1]) # by pop_evaluation[i][1]
        sortednorm = np.argsort(np.linalg.norm(pop_evaluation,axis=1)) # by norm  of pop_evaluation[i]
        
        top0 = self.population[sorted0[:self.reservation_size]]
        top1 = self.population[sorted1[:self.reservation_size]]
        topnorm = self.population[sortednorm[:self.reservation_size]]

        reserved = np.vstack((top0,top1,topnorm))
        remaining_needed = self.pop_size // 2 - len(reserved)

        new_pop = []
        wylosowany=np.random.choice([0,1,2])
        self.WhichIsEvaledMore[wylosowany]+=1
        order_by_rand_fun = [sorted0,sorted1,sortednorm][wylosowany]
        for i in range(remaining_needed):
            new_pop.append(self.population[order_by_rand_fun[i+self.reservation_size]])

        new_pop= np.array(new_pop)
        self.population = np.vstack((reserved,new_pop))
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))


# reserved * ilość funkcji < popsize//2  
model = genetic_tsp_random_fun_eval_reserved(eval,10,100,1)
for i in range(1000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(np.linalg.norm(vec2)<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
print('100 osobnikow -----------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('-------------------------------------')

[321. 350. 329.]
100 osobnikow -----------------------
Min dystans: 267 osiagniety przez: [7 3 4 9 5 6 2 8 1]
Min koszt: 282 osiagniety przez: [9 7 5 6 8 4 1 2 3]
Min norma: 475.47870614781476 osiagniety przez: [7 5 6 2 1 4 3 8 9]
-------------------------------------


Jak widzimy eksperyment zakończył sie sukcesem ponieważ znaleźliśmy optima wszystkich z naszych podgrup, czyli optimum kosztu optimum dystansu i śrenia wartość między nimi.

In [10]:
import numpy as np

def parse_tsp_to_matrix(file_path):
    # Read the TSP file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Initialize variables
    dimension = 0
    edge_weights = []
    
    # Read relevant data from the file
    for line in lines:
        if line.startswith('DIMENSION'):
            dimension = int(line.split(':')[1].strip())
        elif line.startswith('EDGE_WEIGHT_SECTION'):
            break
    
    # Extract the edge weights (from the next lines after EDGE_WEIGHT_SECTION)
    for line in lines[lines.index('EDGE_WEIGHT_SECTION\n') + 1:]:
        if line.strip():  # Avoid empty lines
            edge_weights.extend(map(int, line.split()))
    
    # Now create the symmetric matrix
    matrix = np.zeros((dimension, dimension), dtype=int)
    
    # Fill the upper triangular matrix first
    idx = 0
    for i in range(dimension):
        for j in range(i + 1, dimension):
            matrix[i][j] = edge_weights[idx]
            matrix[j][i] = edge_weights[idx]
            idx += 1
    
    return matrix

odleglosci1 = parse_tsp_to_matrix('randomA100.tsp')
costs1 = parse_tsp_to_matrix('randomB100.tsp')

print(odleglosci1.shape , costs1.shape)


def dist_goal_fun1(perm):
    curr_pos = 0
    dist=0
    for i in perm:
        dist+=odleglosci1[curr_pos][i]
        curr_pos=i
    dist+=odleglosci1[curr_pos][0]
    return dist

def cost_goal_fun1(perm):
    curr_pos = 0
    cost = 0
    for i in perm:
        cost+=costs1[curr_pos][i]
        curr_pos=i
    cost+=costs1[curr_pos][0]
    return cost

def eval1(perm):
    return np.array([dist_goal_fun1(perm),cost_goal_fun1(perm)])


(100, 100) (100, 100)


## eksperyment 3 (wieksze dane z internetu)

In [17]:
model = genetic_tsp_random_fun_eval_reserved(eval1,100,1000,1,100)
for i in range(10000):
    model.step()

print(model.WhichIsEvaledMore)
min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
for perm in model.population:
    vec2= eval1(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(np.linalg.norm(vec2)<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
print('100 osobnikow -----------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('-------------------------------------')

[3306. 3350. 3344.]
100 osobnikow -----------------------
Min dystans: 11498 osiagniety przez: [58 26 53 59 29 14 54 62 22 82 83 41 20 80 60 46 93 76 90 91 45 18 37 10
 21 65 95 87 43 81 40 84 66 28 57  6 13 27 55 99 49 32  2 96 98 24 67  7
 15 31 63 44 89 92 64 48 97 85 78  5 11 23 47 79 39 19  9  4 75 51 25 61
 30 73 36 42 50 68 70 72 86 88 56 52 12 94 16 33  8 17 35 71 34 69 74 38
 77  3  1]
Min koszt: 12317 osiagniety przez: [97 48 64 92 89 44 63 60 80 77 38 74 69 34  8 17 35 71 57 28 94 12 45 91
 90 79 59 53 26 73 30 61 99 49 65  2  5 11 23 47 95 87 32 82 83 41 20 58
 88 86 72 66 50 56 16 33  6 13 27 55 22 62 54 14 29 39 19  9  4 51 25 75
 98 96 78 85 42 36 52 18 37 10 21 43 81 40 84 68 70  1  3 24 67  7 15 31
 46 93 76]
Min norma: 25595.55977117906 osiagniety przez: [97 48 64 92 89 44 63 60 80 77 38 74 69 34 53 26 58 20 41 83 82  8 17 35
 71 57 28 66 98 75  4  9 19 39 79 73 90 91 45 18 37 10 21 32 49 99 55 27
 13  6 33 16 94 12 52 56 88 86 72 70 68 50 65 95 87 43 81 40 84 96 78 8

Można zauważyć że optima nie zostały osiągnięte jednak nie jesteśmy daleko od nich, spójrzmy jeszcze na wyniki ogólne całej populacji żeby móc sprawdzić czy faktycznie odtrzymaliśmy cale spektrum rozwiązań o które nam chodziło

In [24]:
for perm in model.population[:20]:
    vec2= eval1(perm)
    print(f' dystans: {vec2[0]} koszt: {vec2[1]}')

 dystans: 11498 koszt: 51694
 dystans: 11689 koszt: 40144
 dystans: 11780 koszt: 40436
 dystans: 11841 koszt: 40868
 dystans: 11846 koszt: 39703
 dystans: 11937 koszt: 43159
 dystans: 12278 koszt: 35798
 dystans: 12306 koszt: 44627
 dystans: 12376 koszt: 46770
 dystans: 12383 koszt: 36689
 dystans: 12425 koszt: 39486
 dystans: 12488 koszt: 45911
 dystans: 12491 koszt: 45517
 dystans: 12491 koszt: 45517
 dystans: 12585 koszt: 35657
 dystans: 12585 koszt: 35657
 dystans: 12588 koszt: 40491
 dystans: 12599 koszt: 43243
 dystans: 12604 koszt: 38923
 dystans: 12671 koszt: 42730


## wniosek z eksperymentu 3

Przeglądając tylko wyniki kilku pierwszych próbek możemy zaobsrewować coś niepożądanego, mianowicie w końcowej populacji znajdują sie rozwiązania których nie powinno tam nie być tzn rozwiązania zdominowane, czyli takie które osiągają gorsze wyniki od innego osobnika na każdej możliwej funkcji ewaluacyjnej. Jeśli w jakiś sposób udalo by się ograniczyć takie osobniki otrzymali byśmy faktyczne spektrum rozwiązań o które nam chodziło, jednocześnie poprawiając efektywność algorytmu.

# Dywagacje nad problemem generowania zdominowancyh rozwiązań

## Zagadnienie 1 (Czy zdominowane rozwiązania mają wpływ na wynik algorytmu)

Zauważmy że w algorytmach genetycznych przeszukujemy przestrzeń dyskretnie, tzn wybieramy losowych reprezentantów danego wycinka przestrzeni i na ich podstawie oceniamy całą przestrzeń. Ma to znaczenie ponieważ gdybyśmy usunęli z każdej populacji zdominowane osobniki mogło by się okazać że odcieliśmy znaczącą część przestrzeni, ponieważ oceniali byśmy daną mutacje po osobnikach nie reprezentatywnych, w szeczgólności przy znalezieniu optimów lokalnych dla każdej funkcji ciężko było by z nich wyjść, ponieważ "najpierw musilibysmy iść troche pod górkę żeby móc zejść jeszcze niżej"

## Zgadanienie 2 (Jak optymalnie sprawdzać czy rozwiązanie jest zdominowane czy nie)

Niestety jeśli moje rozumowanie jest poprawne nie da sie tego zrobic inaczej niz sprawdzic każdy z każdym czyli w O($ n^2 $) ponieważ załóżmy że nie musimy sprawdzać ze sobą jakieś pary permutacji p i q. Może się okazać że oba są nie dominowane przez żadne inną permutacje jednak p dominuje q.

## Zagadnienie 3 (Jak poradzić sobie z generacją nowej populacji tak aby była jak najmniej zdominowana)

Nie możemy zagwarantować żeby ilości nie zdominowanaych rozwiązań ponieważ w szczególności może istnieć tylko jedno. Jednak chcieli byśmy mieć w naszej populacji jak najmniej zdominowane rozwiązania ponieważ zdominowane rozwiązania heurystycznie dążą do tego samego optimum, a takowe wystarczy że znajdziemy raz. Pomysł jest więc taki zeby do nowej populacji brać jako pierwsze rozwiązania nie zdominowane takie. Nazwijmy cały ich zbiór P1. Następnie jeśli będzie to potrzebne będziemy brali te rozwiązania które były dominowane przez rozwiązania z grupy P1. Takie rozwiązania nawiemy P2. Taki krok będziemy iterować aż znajdziemy całą populacje. Taki podział na grupy nazywa się rozkładem Pareto.

Ale jak to się ma do szukania optimów danych funkcji, zauważmy że gdy rozwiązanie jest w P1 to jest aktualnym optimum w którejś funkcji więc i tak chcieli bysmy żeby znalazło się w populacji, co do rozwiązań z dalszych grup niż P1 są to rozwiązania najbardziej odkrywacze pod względem szukanych przez nas odpowiedzi ponieważ, gdyby coś było zdominowane przez dużo osobników oznacza że w kolejnej populacji dzieci stworzone z tego osobnika prawdopodobnie znowu będą zdominowane przez dzieci osobników które go wcześniej dominowały.

# Idea 3:
Zastosujemy rozkład Pareto na populacji, następnie biorąc do nowej populacji osobniki w kolejności grup w jakich się znajdują aż zapełnimi połowę całej populacji następnie wygenerujemy dzieci od tej populacji w sposób tożsamy z poprzednimi algorytmami, i zaczniemy evaluacje od początku.

In [48]:
import itertools as it
class genetic_tsp_pareto():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        self.WhichIsEvaledMore = np.zeros(3)
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def pareto_fronts(self):
        # wywalenie duplikatów
        self.population = np.unique(self.population,axis=0)
        pareto_fronts=[[] for i in range(len(self.population))]
        how_many_many_dom_u = dict()
        who_do_u_dom = dict()
        evaluated=dict()

        # innit wszystkich dictów
        for perm in self.population:
            how_many_many_dom_u[tuple(perm)]=0
            who_do_u_dom[tuple(perm)]=[]
            evaluated[tuple(perm)]=self.eval_fun(perm)

        # liczenie kto kogo dominuje
        for perm1,perm2 in it.combinations(self.population,2):
            evaluated1 = evaluated[tuple(perm1)]
            evaluated2 = evaluated[tuple(perm2)]
            if( evaluated1[0]>evaluated2[0] and evaluated1[1]>evaluated2[1]):
                how_many_many_dom_u[tuple(perm1)]+=1
                who_do_u_dom[tuple(perm2)].append(tuple(perm1))
            elif(evaluated1[0]<=evaluated2[0] and evaluated1[1]<=evaluated2[1]):
                how_many_many_dom_u[tuple(perm2)]+=1
                who_do_u_dom[tuple(perm1)].append(tuple(perm2))

        selected = 0
        front_num=0
        while (selected != len(self.population)):
            for perm in self.population:
                if(how_many_many_dom_u[tuple(perm)]==0):
                    pareto_fronts[front_num].append(perm)
                    how_many_many_dom_u[tuple(perm)]=-1
                    selected+=1
            for key in pareto_fronts[front_num]:
                for perm in who_do_u_dom[tuple(key)]:
                    how_many_many_dom_u[tuple(perm)]-=1 
            front_num+=1
        return pareto_fronts

    def step(self):
        pareto_fronts = self.pareto_fronts()
        taken=0
        self.population=[]
        for front in pareto_fronts:
            for perm in front:
                if(taken==self.pop_size//2):
                    break
                else:
                    self.population.append(perm)
                    taken+=1
        self.population=np.array(self.population)
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))


# reserved * ilość funkcji < popsize//2  
model = genetic_tsp_pareto(eval,10,100,1)
for i in range(1000):
    model.step()

min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
wazne = model.pareto_fronts()[0]
wazne_sorted = sorted(wazne, key=lambda perm: model.eval_fun(list(perm))[0])
print(len(wazne))
for perm in wazne_sorted:
    vec2= eval(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(np.linalg.norm(vec2)<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
    print(f'dist: {vec2[0]}   cost: {vec2[1]}')


17
dist: 267   cost: 509
dist: 269   cost: 500
dist: 274   cost: 451
dist: 276   cost: 429
dist: 297   cost: 415
dist: 309   cost: 381
dist: 316   cost: 380
dist: 324   cost: 348
dist: 349   cost: 334
dist: 388   cost: 316
dist: 433   cost: 302
dist: 468   cost: 301
dist: 478   cost: 298
dist: 495   cost: 292
dist: 512   cost: 290
dist: 523   cost: 289
dist: 545   cost: 282


In [49]:
print('100 osobnikow -----------------------')
print(f'Min dystans: {min_dist} osiagniety przez: {min_dist_perm}')
print(f'Min koszt: {min_cost} osiagniety przez: {min_cost_perm}')
print(f'Min norma: {min_norm} osiagniety przez: {min_norm_perm}')
print('-------------------------------------')

100 osobnikow -----------------------
Min dystans: 267 osiagniety przez: [1 4 9 5 6 2 8 3 7]
Min koszt: 282 osiagniety przez: [3 2 1 4 8 6 5 7 9]
Min norma: 475.47870614781476 osiagniety przez: [7 5 6 2 1 4 3 8 9]
-------------------------------------


In [56]:
import itertools as it
class genetic_tsp_pareto():
    def __init__(self,eval_fun,perm_size,pop_size,mutation_propabilty):
        self.eval_fun = eval_fun
        self.perm_size=perm_size
        self.pop_size = pop_size
        self.mutation_propabilty = mutation_propabilty
        self.WhichIsEvaledMore = np.zeros(3)
        # + 1 bo zawsze zaczynamy od 0 więc defakto mamy permutacje tylko 9 elemntów 
        self.population = np.array([np.random.permutation(perm_size-1) + 1 for i in range(pop_size)])
        self.children = self.population.copy()

    # weźmiemy standardowa mutacje dla tego problemu czyli transpozycje tablicy elemntów (losowanie 2 indexów i odwrócenie kolejności wszystkich elementów pomiędzy nimi)
    def mutation(self):
        for child_perm in self.children:
            if(self.mutation_propabilty>np.random.uniform()):
                a = np.random.choice(self.perm_size,2,False)
                i,j = a.min(),a.max()
                child_perm[i:j+1]=child_perm[i:j+1][::-1]
                

    def cross_over(self,parent1,parent2):
        cutoff_1, cutoff_2 = np.sort(np.random.choice(self.perm_size, 2, replace=False))

        def PMX_one_offspring(p1, p2):
            offspring = np.zeros(len(p1), dtype=p1.dtype)

            # Copy the mapping section (middle) from parent1
            offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

            # copy the rest from parent2 (provided it's not already there
            for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
                candidate = p2[i]
                while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                    candidate = p2[np.where(p1 == candidate)[0][0]]
                offspring[i] = candidate
            return offspring

        offspring1 = PMX_one_offspring(parent1, parent2)
        #offspring2 = PMX_one_offspring(parent2, parent1)

        return offspring1

    def gen_kids(self):
        children = []
        for i in range(self.pop_size-len(self.population)):
            p1_index,p2_index = np.random.choice(len(self.population),2,replace=False)
            parent1,parent2 = self.population[p1_index],self.population[p2_index]
            child = self.cross_over(parent1,parent2)
            children.append(child)
        self.children = np.array(children)

    def pareto_fronts(self):
        # wywalenie duplikatów
        self.population = np.unique(self.population,axis=0)
        pareto_fronts=[[] for i in range(len(self.population))]
        how_many_many_dom_u = dict()
        who_do_u_dom = dict()
        evaluated=dict()

        # innit wszystkich dictów
        for perm in self.population:
            how_many_many_dom_u[tuple(perm)]=0
            who_do_u_dom[tuple(perm)]=[]
            evaluated[tuple(perm)]=self.eval_fun(perm)

        # liczenie kto kogo dominuje
        for perm1,perm2 in it.combinations(self.population,2):
            evaluated1 = evaluated[tuple(perm1)]
            evaluated2 = evaluated[tuple(perm2)]
            if( evaluated1[0]>evaluated2[0] and evaluated1[1]>evaluated2[1]):
                how_many_many_dom_u[tuple(perm1)]+=1
                who_do_u_dom[tuple(perm2)].append(tuple(perm1))
            elif(evaluated1[0]<=evaluated2[0] and evaluated1[1]<=evaluated2[1]):
                how_many_many_dom_u[tuple(perm2)]+=1
                who_do_u_dom[tuple(perm1)].append(tuple(perm2))

        selected = 0
        front_num=0
        while (selected != len(self.population)):
            for perm in self.population:
                if(how_many_many_dom_u[tuple(perm)]==0):
                    pareto_fronts[front_num].append(perm)
                    how_many_many_dom_u[tuple(perm)]=-1
                    selected+=1
            for key in pareto_fronts[front_num]:
                for perm in who_do_u_dom[tuple(key)]:
                    how_many_many_dom_u[tuple(perm)]-=1 
            front_num+=1
        return pareto_fronts

    def step(self):
        pareto_fronts = self.pareto_fronts()
        taken=0
        self.population=[]
        for front in pareto_fronts:
            for perm in front:
                if(taken==self.pop_size//2):
                    break
                else:
                    self.population.append(perm)
                    taken+=1
        self.population=np.array(self.population)
        self.gen_kids()
        self.mutation()
        self.population = np.vstack((self.population,self.children))

# reserved * ilość funkcji < popsize//2  
model = genetic_tsp_pareto(eval1,100,1000,1)
for i in range(1000):
    model.step()

min_dist=np.inf
min_dist_perm=0
min_cost=np.inf
min_cost_perm=0
min_norm=np.inf
min_norm_perm=0
wazne = model.pareto_fronts()[0]
wazne_sorted = sorted(wazne, key=lambda perm: model.eval_fun(list(perm))[0])
print(len(wazne))
for perm in wazne_sorted:
    vec2= eval1(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(np.linalg.norm(vec2)<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
    print(f'dist: {vec2[0]}   cost: {vec2[1]}')


KeyboardInterrupt: 

In [52]:
print(f'zbiór rozwiązań niezdominowanaych długości {len(wazne)}----------------------------')
for perm in wazne_sorted:
    vec2= eval1(perm)
    if(vec2[0]<min_dist):
        min_dist=vec2[0]
        min_dist_perm=perm
    if(vec2[1]<min_cost):
        min_cost=vec2[1]
        min_cost_perm=perm
    if(np.linalg.norm(vec2)<min_norm):
        min_norm=np.linalg.norm(vec2)
        min_norm_perm=perm
    print(f'dist: {vec2[0]}   cost: {vec2[1]}')

zbiór rozwiązań niezdominowanaych długości 33----------------------------
dist: 77221   cost: 162046
dist: 79732   cost: 161755
dist: 81538   cost: 152974
dist: 81880   cost: 152677
dist: 81941   cost: 148257
dist: 83084   cost: 130441
dist: 83496   cost: 126699
dist: 83927   cost: 125974
dist: 85496   cost: 123432
dist: 87589   cost: 114793
dist: 89040   cost: 107218
dist: 100722   cost: 105838
dist: 100741   cost: 105080
dist: 103298   cost: 104642
dist: 103310   cost: 104489
dist: 103629   cost: 101975
dist: 105242   cost: 98443
dist: 107560   cost: 97859
dist: 107644   cost: 92014
dist: 108478   cost: 89130
dist: 110672   cost: 88981
dist: 116563   cost: 88935
dist: 119935   cost: 88710
dist: 122291   cost: 87938
dist: 124503   cost: 87070
dist: 128051   cost: 86980
dist: 128327   cost: 83955
dist: 144441   cost: 80251
dist: 159675   cost: 80178
dist: 159699   cost: 77676
dist: 168803   cost: 76854
dist: 174106   cost: 76773
dist: 176283   cost: 74185


# wnioski i podsumowanie wyników

Wyniki chociaż nie są zadowalające w sensie uzysknaych wartości są  w porównaniu do tych osiągniętych które są w odpowiedziach, generują jednak zawsze zbiór nie zdominowanych w swojej populacji rozwiązań, czyli algorytm daje nam pewną gamę rozwiazań z czego każdy jest pewnym optimum, a o to nam chodziło

W porównaniu do pozostałych algorytmów podział pareto wyróżnił się tym że stawiał na róznorodność bardziej niż szukanie poszczególnych optimów funkcji.
Algorytm zarezerwowanych miejsc natomiast pomimo swojej umiarkowanie szybkiej zbiezności do minimów lokalnych osiągną znacząoco lepsze ptima poszczególnych funkcji, przez co mozna zakładać że pomimo że zbiór rozwiązań niezależnych w tamtym przypadku nie był by zbyt duży to był by on 'optymalniejszy' tzn zdominował by znaczącą część wygenerowanych w końcowym algorytmie.

## Z czego mogą wynikać tak słabe szukanie optimów poszczególnych funkcji?

Tak jak już pisałem w rozkładzie pareto najbardziej ceniona była różnorodność, przez co do danego optimum lokalnego dążyło mniej osobników więc ciężko było żeby losowo trafić na optymalizującą mutacje.

## Perspektywy Rozwoju

Z analizy poprzedniego rozwiązania można wywnioskować że rezerwacja miejsc była dobrym sposobem na pogłębianie ekstremów, by ulepszyć algorytm oparty na rozkładzie pareto, można by połączyć oba rozwiązania. To znaczy dalej promować różnorodność jednak tym razem mieć również zarezerwowane miejsca dla kilku pierwszych optimów każdej funkcji celu, by dla każdej z nich znalezinoe minimum w rozwiązaniach było jak kolwiek wiarygodne. Nastepnie można wprowadzić pewną modyfikacje 'warunek stop-u' jeżeli przez iles generacji nie udało nam się poprawić optimum danej funckji można rezerwacje miejsc przerzucić z optium funkcji celu do np. rozwiązań 2 względem tej funkcji niezależnego rozwiązania, itd aż wszystkie rozwiązania niezalezne 'znajdą swoje optima' bądź przestaną być niezależne.